# Ad Stock Analysis - Data Pull

This notebook pulls data for ad stock model analysis, focusing on power players (top vendors and users).

**Key Features:**
- Identifies power vendors (80% of ad spend) and power users (80% of purchases)
- Uses efficient CTE-based sampling at Snowflake level
- Standardizes all IDs for consistent joining
- Saves data as Parquet files for efficient processing

In [1]:
import os
import textwrap
from datetime import date, timedelta, datetime
from pathlib import Path
import warnings
import pandas as pd
from dotenv import load_dotenv
import snowflake.connector
from tqdm import tqdm
import json
import gc

# Suppress pandas SQLAlchemy warning
warnings.filterwarnings(
    'ignore',
    category=UserWarning,
    message='pandas only supports SQLAlchemy connectable.*'
)

## 1. Configuration

In [2]:
# Load environment variables
load_dotenv()

# Time windows - 365 day window for comprehensive ad stock analysis
ANALYSIS_END_DATE = date(2025, 9, 2)
TOTAL_PULL_DAYS = 365
ANALYSIS_START_DATE = ANALYSIS_END_DATE - timedelta(days=TOTAL_PULL_DAYS)

# Sampling parameters - focused on power players
SAMPLING_FRACTION = 0.001  # 0.1% of users for initial testing
POWER_VENDOR_PERCENTILE = 80  # Top vendors accounting for 80% of spend
POWER_USER_PERCENTILE = 80    # Top users accounting for 80% of purchases

# Output paths
DATA_DIR = Path('./data')
DATA_DIR.mkdir(exist_ok=True)

print("Configuration:")
print(f"  Analysis period: {ANALYSIS_START_DATE} to {ANALYSIS_END_DATE}")
print(f"  Total days: {TOTAL_PULL_DAYS}")
print(f"  Sampling fraction: {SAMPLING_FRACTION:.2%}")
print(f"  Power vendor percentile: {POWER_VENDOR_PERCENTILE}%")
print(f"  Power user percentile: {POWER_USER_PERCENTILE}%")

Configuration:
  Analysis period: 2024-09-02 to 2025-09-02
  Total days: 365
  Sampling fraction: 0.10%
  Power vendor percentile: 80%
  Power user percentile: 80%


## 2. Snowflake Connection

In [3]:
try:
    conn = snowflake.connector.connect(
        user=os.getenv('SNOWFLAKE_USER'),
        password=os.getenv('SNOWFLAKE_PASSWORD'),
        account=os.getenv('SNOWFLAKE_ACCOUNT'),
        warehouse=os.getenv('SNOWFLAKE_WAREHOUSE', 'COMPUTE_WH'),
        database='INCREMENTALITY',
        schema='INCREMENTALITY_RESEARCH'
    )
    print("[SUCCESS] Snowflake connection established.")
except Exception as e:
    print(f"[FAILURE] Could not connect to Snowflake: {e}")
    conn = None

[SUCCESS] Snowflake connection established.


## 3. Identify Power Players

First, we identify the top vendors and users who drive the majority of platform activity.

In [4]:
def identify_power_vendors(conn, start_date: str, end_date: str, percentile: int = 80) -> pd.DataFrame:
    """Identify top vendors accounting for X% of platform ad activity."""
    print(f"\nIdentifying power vendors (top {percentile}% of activity)...")
    
    query = textwrap.dedent(f"""
        WITH VENDOR_ACTIVITY AS (
            SELECT
                LOWER(TO_VARCHAR(ar.VENDOR_ID, 'HEX')) AS VENDOR_ID,
                COUNT(DISTINCT ar.AUCTION_ID) AS total_bids,
                SUM(CASE WHEN ar.IS_WINNER THEN 1 ELSE 0 END) AS winning_bids,
                COUNT(DISTINCT ar.CAMPAIGN_ID) AS total_campaigns
            FROM AUCTIONS_RESULTS ar
            WHERE ar.CREATED_AT BETWEEN '{start_date}' AND '{end_date}'
            GROUP BY 1
        ),
        VENDOR_RANKED AS (
            SELECT
                VENDOR_ID,
                total_bids,
                winning_bids,
                total_campaigns,
                SUM(winning_bids) OVER (ORDER BY winning_bids DESC) AS cumulative_wins,
                SUM(winning_bids) OVER () AS total_wins
            FROM VENDOR_ACTIVITY
        )
        SELECT
            VENDOR_ID,
            total_bids,
            winning_bids,
            total_campaigns,
            ROUND(100.0 * cumulative_wins / total_wins, 2) AS cumulative_pct
        FROM VENDOR_RANKED
        WHERE cumulative_pct <= {percentile}
        ORDER BY winning_bids DESC
    """)
    
    df = pd.read_sql(query, conn)
    print(f"  Found {len(df):,} power vendors accounting for top {percentile}% of activity")
    print(f"  Top vendor has {df.iloc[0]['winning_bids']:,} winning bids")
    return df

In [5]:
def identify_power_users(conn, start_date: str, end_date: str, percentile: int = 80) -> pd.DataFrame:
    """Identify top users accounting for X% of platform purchases."""
    print(f"\nIdentifying power users (top {percentile}% of purchases)...")
    
    query = textwrap.dedent(f"""
        WITH USER_ACTIVITY AS (
            SELECT
                USER_ID,
                COUNT(DISTINCT PURCHASE_ID) AS total_purchases,
                SUM(QUANTITY * UNIT_PRICE) AS total_revenue,
                COUNT(DISTINCT PRODUCT_ID) AS distinct_products
            FROM PURCHASES
            WHERE PURCHASED_AT BETWEEN '{start_date}' AND '{end_date}'
            GROUP BY 1
        ),
        USER_RANKED AS (
            SELECT
                USER_ID,
                total_purchases,
                total_revenue,
                distinct_products,
                SUM(total_revenue) OVER (ORDER BY total_revenue DESC) AS cumulative_revenue,
                SUM(total_revenue) OVER () AS platform_revenue
            FROM USER_ACTIVITY
        )
        SELECT
            USER_ID,
            total_purchases,
            total_revenue,
            distinct_products,
            ROUND(100.0 * cumulative_revenue / platform_revenue, 2) AS cumulative_pct
        FROM USER_RANKED
        WHERE cumulative_pct <= {percentile}
        ORDER BY total_revenue DESC
        LIMIT 50000  -- Cap for memory efficiency
    """)
    
    df = pd.read_sql(query, conn)
    print(f"  Found {len(df):,} power users accounting for top {percentile}% of revenue")
    print(f"  Top user has ${df.iloc[0]['total_revenue']:,.2f} in purchases")
    return df

In [ ]:
# Identify power players
if conn:
    start_date_str = ANALYSIS_START_DATE.strftime('%Y-%m-%d')
    end_date_str = ANALYSIS_END_DATE.strftime('%Y-%m-%d')
    
    power_vendors = identify_power_vendors(conn, start_date_str, end_date_str, POWER_VENDOR_PERCENTILE)
    power_users = identify_power_users(conn, start_date_str, end_date_str, POWER_USER_PERCENTILE)
    
    # Save power player lists
    power_vendors.to_parquet(DATA_DIR / 'power_vendors.parquet', index=False)
    power_users.to_parquet(DATA_DIR / 'power_users.parquet', index=False)
    
    print("\nPower players identified and saved.")


Identifying power vendors (top 80% of activity)...


## 4. Build CTE for Power Player Sampling

In [ ]:
def build_power_player_cte(start_date: str, end_date: str,
                           power_vendor_percentile: int = 80,
                           power_user_percentile: int = 80,
                           sampling_fraction: float = 0.001) -> str:
    """Build CTE that identifies power players inline without passing IDs through VALUES clause."""

    total_buckets = 10000
    selection_threshold = int(total_buckets * sampling_fraction)

    return textwrap.dedent(f"""
        WITH POWER_VENDORS AS (
            -- Re-calculate power vendors inline to avoid SQL length limits
            WITH VENDOR_ACTIVITY AS (
                SELECT
                    LOWER(TO_VARCHAR(ar.VENDOR_ID, 'HEX')) AS VENDOR_ID,
                    SUM(CASE WHEN ar.IS_WINNER THEN 1 ELSE 0 END) AS winning_bids
                FROM AUCTIONS_RESULTS ar
                WHERE ar.CREATED_AT BETWEEN '{start_date}' AND '{end_date}'
                GROUP BY 1
            ),
            VENDOR_RANKED AS (
                SELECT
                    VENDOR_ID,
                    winning_bids,
                    SUM(winning_bids) OVER (ORDER BY winning_bids DESC) AS cumulative_wins,
                    SUM(winning_bids) OVER () AS total_wins
                FROM VENDOR_ACTIVITY
            )
            SELECT VENDOR_ID
            FROM VENDOR_RANKED
            WHERE (100.0 * cumulative_wins / NULLIF(total_wins, 0)) <= {power_vendor_percentile}
        ),
        POWER_USERS AS (
            -- Re-calculate power users inline to avoid SQL length limits
            WITH USER_ACTIVITY AS (
                SELECT
                    USER_ID,
                    SUM(QUANTITY * UNIT_PRICE) AS total_revenue
                FROM PURCHASES
                WHERE PURCHASED_AT BETWEEN '{start_date}' AND '{end_date}'
                GROUP BY 1
            ),
            USER_RANKED AS (
                SELECT
                    USER_ID,
                    total_revenue,
                    SUM(total_revenue) OVER (ORDER BY total_revenue DESC) AS cumulative_revenue,
                    SUM(total_revenue) OVER () AS platform_revenue
                FROM USER_ACTIVITY
            )
            SELECT USER_ID
            FROM USER_RANKED
            WHERE (100.0 * cumulative_revenue / NULLIF(platform_revenue, 0)) <= {power_user_percentile}
            LIMIT 50000  -- Safety limit
        ),
        SAMPLED_USERS AS (
            -- Combine power users with sampled regular users
            SELECT USER_ID FROM POWER_USERS
            UNION
            -- Sample additional users based on hash
            SELECT DISTINCT au.OPAQUE_USER_ID AS USER_ID
            FROM AUCTIONS_USERS au
            WHERE au.CREATED_AT BETWEEN '{start_date}' AND '{end_date}'
              AND MOD(ABS(HASH(au.OPAQUE_USER_ID)), {total_buckets}) < {selection_threshold}
              AND au.OPAQUE_USER_ID NOT IN (SELECT USER_ID FROM POWER_USERS)
        )
    """)

## 5. Extract Core Event Data

In [ ]:
def extract_auctions_users(conn, start_date: str, end_date: str, cte: str) -> pd.DataFrame:
    """Extract AUCTIONS_USERS table with power player filtering."""
    print("\nExtracting AUCTIONS_USERS...")
    
    query = cte + textwrap.dedent(f"""
        SELECT
            LOWER(TO_VARCHAR(au.AUCTION_ID, 'HEX')) AS AUCTION_ID,
            au.OPAQUE_USER_ID AS USER_ID,
            au.CREATED_AT,
            DATE(au.CREATED_AT) AS auction_date,
            HOUR(au.CREATED_AT) AS auction_hour,
            DAYOFWEEK(au.CREATED_AT) AS auction_dow,
            WEEKOFYEAR(au.CREATED_AT) AS auction_week
        FROM AUCTIONS_USERS au
        JOIN SAMPLED_USERS s ON au.OPAQUE_USER_ID = s.USER_ID
        WHERE au.CREATED_AT BETWEEN '{start_date}' AND '{end_date}'
        ORDER BY au.OPAQUE_USER_ID, au.CREATED_AT
    """)
    
    df = pd.read_sql(query, conn)
    df.columns = [x.lower() for x in df.columns]
    print(f"  Extracted {len(df):,} auction records")
    return df

In [ ]:
def extract_impressions(conn, start_date: str, end_date: str, cte: str) -> pd.DataFrame:
    """Extract IMPRESSIONS table focusing on power players."""
    print("\nExtracting IMPRESSIONS...")
    
    query = cte + textwrap.dedent(f"""
        SELECT
            i.INTERACTION_ID AS impression_id,
            LOWER(REPLACE(i.AUCTION_ID, '-', '')) AS AUCTION_ID,
            LOWER(TRIM(i.PRODUCT_ID)) AS PRODUCT_ID,
            i.USER_ID,
            LOWER(REPLACE(i.CAMPAIGN_ID, '-', '')) AS CAMPAIGN_ID,
            LOWER(REPLACE(i.VENDOR_ID, '-', '')) AS VENDOR_ID,
            i.OCCURRED_AT AS impression_time
        FROM IMPRESSIONS i
        WHERE i.OCCURRED_AT BETWEEN '{start_date}' AND '{end_date}'
          AND (
              i.USER_ID IN (SELECT USER_ID FROM SAMPLED_USERS)
              OR LOWER(REPLACE(i.VENDOR_ID, '-', '')) IN (SELECT VENDOR_ID FROM POWER_VENDORS)
          )
    """)
    
    df = pd.read_sql(query, conn)
    df.columns = [x.lower() for x in df.columns]
    print(f"  Extracted {len(df):,} impression records")
    return df

In [ ]:
def extract_clicks(conn, start_date: str, end_date: str, cte: str) -> pd.DataFrame:
    """Extract CLICKS table focusing on power players."""
    print("\nExtracting CLICKS...")
    
    query = cte + textwrap.dedent(f"""
        SELECT
            c.INTERACTION_ID AS click_id,
            LOWER(REPLACE(c.AUCTION_ID, '-', '')) AS AUCTION_ID,
            LOWER(TRIM(c.PRODUCT_ID)) AS PRODUCT_ID,
            c.USER_ID,
            LOWER(REPLACE(c.CAMPAIGN_ID, '-', '')) AS CAMPAIGN_ID,
            LOWER(REPLACE(c.VENDOR_ID, '-', '')) AS VENDOR_ID,
            c.OCCURRED_AT AS click_time
        FROM CLICKS c
        WHERE c.OCCURRED_AT BETWEEN '{start_date}' AND '{end_date}'
          AND (
              c.USER_ID IN (SELECT USER_ID FROM SAMPLED_USERS)
              OR LOWER(REPLACE(c.VENDOR_ID, '-', '')) IN (SELECT VENDOR_ID FROM POWER_VENDORS)
          )
    """)
    
    df = pd.read_sql(query, conn)
    df.columns = [x.lower() for x in df.columns]
    print(f"  Extracted {len(df):,} click records")
    return df

In [ ]:
def extract_purchases(conn, start_date: str, end_date: str, cte: str) -> pd.DataFrame:
    """Extract PURCHASES table for sampled users."""
    print("\nExtracting PURCHASES...")
    
    query = cte + textwrap.dedent(f"""
        SELECT
            p.PURCHASE_ID,
            p.PURCHASED_AT AS purchase_time,
            LOWER(TRIM(p.PRODUCT_ID)) AS PRODUCT_ID,
            p.QUANTITY,
            p.UNIT_PRICE,
            p.USER_ID,
            p.PURCHASE_LINE,
            (p.QUANTITY * p.UNIT_PRICE) AS revenue
        FROM PURCHASES p
        JOIN SAMPLED_USERS s ON p.USER_ID = s.USER_ID
        WHERE p.PURCHASED_AT BETWEEN '{start_date}' AND '{end_date}'
    """)
    
    df = pd.read_sql(query, conn)
    df.columns = [x.lower() for x in df.columns]
    print(f"  Extracted {len(df):,} purchase records")
    return df

In [ ]:
def extract_catalog(conn, product_ids: set) -> pd.DataFrame:
    """Extract CATALOG for relevant products."""
    print("\nExtracting CATALOG...")
    
    # Convert product IDs to SQL-friendly format (limit to prevent SQL too long)
    product_ids_str = "','" .join(list(product_ids)[:50000])
    
    query = textwrap.dedent(f"""
        SELECT
            LOWER(TRIM(c.PRODUCT_ID)) AS PRODUCT_ID,
            c.NAME AS product_name,
            c.PRICE AS catalog_price,
            c.ACTIVE AS is_active,
            c.IS_DELETED,
            SPLIT_PART(ARRAY_TO_STRING(FILTER(c.CATEGORIES, x -> x LIKE 'brand#%%'), ''), '#', 2) AS BRAND,
            SPLIT_PART(ARRAY_TO_STRING(FILTER(c.CATEGORIES, x -> x LIKE 'department#%%'), ''), '#', 2) AS DEPARTMENT_ID,
            SPLIT_PART(ARRAY_TO_STRING(FILTER(c.CATEGORIES, x -> x LIKE 'category#%%'), ''), '#', 2) AS CATEGORY_ID
        FROM CATALOG c
        WHERE LOWER(TRIM(c.PRODUCT_ID)) IN ('{product_ids_str}')
    """)
    
    df = pd.read_sql(query, conn)
    df.columns = [x.lower() for x in df.columns]
    print(f"  Extracted {len(df):,} catalog records")
    return df

## 6. Main Data Extraction Pipeline

In [ ]:
if conn and 'power_vendors' in locals() and 'power_users' in locals():
    print("="*80)
    print("STARTING DATA EXTRACTION PIPELINE FOR AD STOCK ANALYSIS")
    print("="*80)
    
    # Create timestamp for this extraction run
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    # Convert dates to strings for SQL
    start_date_str = ANALYSIS_START_DATE.strftime('%Y-%m-%d')
    end_date_str = ANALYSIS_END_DATE.strftime('%Y-%m-%d')
    
    # Build the CTE for power player filtering - now passing dates and percentiles instead of DataFrames
    power_player_cte = build_power_player_cte(
        start_date_str, 
        end_date_str,
        POWER_VENDOR_PERCENTILE,
        POWER_USER_PERCENTILE,
        SAMPLING_FRACTION
    )
    
    # Extract main event tables
    print("\n--- Extracting event data for power players ---")
    auctions_users = extract_auctions_users(conn, start_date_str, end_date_str, power_player_cte)
    impressions = extract_impressions(conn, start_date_str, end_date_str, power_player_cte)
    clicks = extract_clicks(conn, start_date_str, end_date_str, power_player_cte)
    purchases = extract_purchases(conn, start_date_str, end_date_str, power_player_cte)
    
    # Collect all product IDs
    all_product_ids = set()
    all_product_ids.update(impressions['product_id'].unique())
    all_product_ids.update(clicks['product_id'].unique())
    all_product_ids.update(purchases['product_id'].unique())
    
    # Extract catalog
    catalog = extract_catalog(conn, all_product_ids)
    
    # Close connection
    conn.close()
    print("\n[SUCCESS] Snowflake connection closed")
else:
    print("[ERROR] Missing connection or power player data")

## 7. Data Validation

In [ ]:
def validate_data(auctions_users, impressions, clicks, purchases, catalog):
    """Validate data integrity and print summary statistics."""
    
    print("\n" + "="*50)
    print("DATA VALIDATION SUMMARY")
    print("="*50)
    
    # Basic counts
    print("\nRecord Counts:")
    print(f"  Auctions:    {len(auctions_users):,}")
    print(f"  Impressions: {len(impressions):,}")
    print(f"  Clicks:      {len(clicks):,}")
    print(f"  Purchases:   {len(purchases):,}")
    print(f"  Catalog:     {len(catalog):,}")
    
    # Unique counts
    print("\nUnique Entities:")
    print(f"  Users:       {auctions_users['user_id'].nunique():,}")
    print(f"  Vendors:     {impressions['vendor_id'].nunique():,}")
    print(f"  Products:    {len(all_product_ids):,}")
    
    # Power player coverage
    power_vendor_ids = set(power_vendors['vendor_id'].values)
    power_user_ids = set(power_users['user_id'].values)
    
    impression_vendors = set(impressions['vendor_id'].unique())
    impression_users = set(impressions['user_id'].unique())
    
    print("\nPower Player Coverage:")
    print(f"  Power vendors in impressions: {len(impression_vendors & power_vendor_ids):,}/{len(power_vendor_ids):,}")
    print(f"  Power users in impressions: {len(impression_users & power_user_ids):,}/{len(power_user_ids):,}")
    
    # Conversion funnel
    print("\nConversion Funnel:")
    print(f"  Impressions:      {len(impressions):,}")
    if len(impressions) > 0:
        print(f"  Clicks:           {len(clicks):,} ({len(clicks)/len(impressions)*100:.1f}% CTR)")
    print(f"  Purchase Events:  {len(purchases):,}")

if 'auctions_users' in locals():
    validate_data(auctions_users, impressions, clicks, purchases, catalog)

## 8. Save Data

In [ ]:
if 'impressions' in locals():
    print("\n--- Saving data for ad stock analysis ---")
    output_dir = DATA_DIR
    output_dir.mkdir(parents=True, exist_ok=True)
    
    datasets = [
        ("auctions_users", auctions_users),
        ("impressions", impressions),
        ("clicks", clicks),
        ("purchases", purchases),
        ("catalog", catalog)
    ]
    
    for name, df in tqdm(datasets, desc="Saving Parquet files"):
        path = output_dir / f"{name}_adstock.parquet"
        df.to_parquet(path, index=False)
    
    print("\nAll data files saved successfully.")
    
    # Create metadata file
    metadata = {
        'timestamp': timestamp,
        'analysis_start_date': start_date_str,
        'analysis_end_date': end_date_str,
        'total_days': TOTAL_PULL_DAYS,
        'sampling_fraction': SAMPLING_FRACTION,
        'power_vendor_percentile': POWER_VENDOR_PERCENTILE,
        'power_user_percentile': POWER_USER_PERCENTILE,
        'n_power_vendors': len(power_vendors),
        'n_power_users': len(power_users),
        'total_products': len(all_product_ids),
        'row_counts': {name: len(df) for name, df in datasets}
    }
    
    metadata_path = output_dir / f"metadata_adstock.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved metadata to {metadata_path.name}")
    
    print("\n" + "="*80)
    print("DATA EXTRACTION COMPLETE")
    print("="*80)
    print(f"All data saved to {output_dir}/")
    print(f"Ready for ad stock feature engineering in notebook 02")

## 9. Memory Cleanup

In [ ]:
# Clean up memory
if 'auctions_users' in locals():
    del auctions_users, impressions, clicks, purchases, catalog
    gc.collect()
    print("✓ Memory cleared")